## Progress report 3 notebook

This notebook loads and parses a sample of the XML data used in the project. A general report that includes more information about the project is available in the [project report](../docs/progress_report.md).

This notebook continues [progress_report_1.ipynb](progress_report_1.ipynb) and [progress_report_2.ipynb](progress_report_2.ipynb), integrating improvements, corrections, and new functionality. This is the *existing* option for report 3, renamed for easy access to and comparison of the three stages.

We have changed the method in the following two ways:

### 1. Replace syllabification with C(C)/V decomposition

Instead of decomposing the rhyme zone into syllables, we decompose it into sequences of vowels and consonant clusters (which may be single consonants). This modification was adopted because segments that should be regarded as matching for rhyme analysis purposes may belong to different syllables, which means that syllabification could compromise their identification. For example:

<table>
    <tr><th>Word</th><th>Phonetic</th><th>|</th><th colspan="5">Syllables</th><th>|</th><th colspan="11">C(C)/V decomposition</th></tr>
    <tr>
        <td>вы́бора</td>
        <td>vIbara</td>
        <td>|</td>
        <td>vI</td>
        <td>-</td>
        <td>ba</td>
        <td>-</td>
        <td>ra</td>
        <td>|</td>
        <td>v</td>
        <td>-</td>
        <td>I</td>
        <td>-</td>
        <td>b</td>
        <td>-</td>
        <td>a</td>
        <td>-</td>
        <td>r</td>
        <td>-</td>
        <td>a</td>
    </tr>
    <tr>
        <td>вы́борка</td>
        <td>vIbarka</td>
        <td>|</td>
        <td>vI</td>
        <td>-</td>
        <td>bar</td>
        <td>-</td>
        <td>ka</td>
        <td>|</td>
        <td>v</td>
        <td>-</td>
        <td>I</td>
        <td>-</td>
        <td>b</td>
        <td>-</td>
        <td>a</td>
        <td>-</td>
        <td>rk</td>
        <td>-</td>
        <td>a</td>
    </tr>
</table>
    

### 2. Identify rhymes by segments, rather than distinctive features

Our original assumption was that after decomposing syllables into onset, nucleus, and coda we could decompose those parts into segments (the nucleus is always present and monosegmental; the onset and coda are optional and potentially polysegmental). This turns out not to be helpful in situations where a monosegmental component might have to be compared to a polysegmental one, since it isn’t clear where the single segment should be aligned. We may reevaluate this decision later (perhaps feature-level comparison will prove useful in cases of isosegmental columns; perhaps we will introduce alignment logic to assign the segments in anisosegmental columns), and we will nonetheless continue to use phonetic distinctive features in evaluating and analyzing rhyming lines, even if not for identifying them.

## Sample data

The sample dataset consists of multiple stanzas from a long poem. An artificial one-stanza extract looks like the following:

```xml
<poem>
    <stanza stanzaNo="001">
        <line lineNo="001">"Мой дядя самых честных пр<stress>а</stress>вил,</line>
        <line lineNo="002">Когда не в шутку занем<stress>о</stress>г,</line>
        <line lineNo="003">Он уважать себя заст<stress>а</stress>вил</line>
        <line lineNo="004">И лучше выдумать не м<stress>о</stress>г.</line>
        <line lineNo="005">Его пример другим на<stress>у</stress>ка;</line>
        <line lineNo="006">Но, боже мой, какая ск<stress>у</stress>ка</line>
        <line lineNo="007">С больным сидеть и день и н<stress>о</stress>чь,</line>
        <line lineNo="008">Не отходя ни шагу пр<stress>о</stress>чь!</line>
        <line lineNo="009">Какое низкое ков<stress>а</stress>рство</line>
        <line lineNo="010">Полу-живого забавл<stress>я</stress>ть,</line>
        <line lineNo="011">Ему подушки поправл<stress>я</stress>ть,</line>
        <line lineNo="012">Печально подносить лек<stress>а</stress>рство,</line>
        <line lineNo="013">Вздыхать и думать про себ<stress>я</stress>:</line>
        <line lineNo="014">Когда же чорт возьмет теб<stress>я</stress>!"</line>
    </stanza>
</poem>   
```

## Reload libraries each time, since we’re tinkering with them

In [1]:
%load_ext autoreload
%autoreload 2

## Load libraries

In [2]:
from xml.dom import pulldom  # parse input XML
from xml.dom.minidom import Document  # construct output XML
import numpy as np
import pandas as pd
import regex as re
from cyr2phon import cyr2phon  # custom package

## Class and variables for parsing input XML

In [3]:
class Stack(list):  # keep track of open nodes while constructing XML output
    def push(self, item):
        self.append(item)

    def peek(self):  
        return self[-1]


open_elements = Stack()
WS_RE = re.compile(r'\s+')  # normalize white space in output

## Function to parse the XML

Returns a list of lists, with stanza number, line number, and `<line>` element for each line. We use the light-weight *xml.dom.pulldom* library to parse the input XML and *xml.dom.minidom* to construct the lines as simplified XML, removing elements we don’t care about, such as `<latin>` and `<italic>`, before serializing them to the output. (We actually do care about `<latin>`, but we are ignoring it temporarily, and we’ll return to it at a later stage in the project.)

In [4]:
def process(input_xml):
    stanzaNo = 0
    lineNo = 0
    inline = 0  # flag to control behavior inside and outside lines
    result = []  # array of arrays, one per line, with stanzaNo, lineNo, and serialized XML
    doc = pulldom.parse(input_xml)
    for event, node in doc:
        if event == pulldom.START_ELEMENT and node.localName == 'stanza':
            stanzaNo = node.getAttribute("stanzaNo")
        elif event == pulldom.START_ELEMENT and node.localName == 'line':
            d = Document()  # each line is an output XML document
            open_elements.push(d)  # document node
            lineNo = node.getAttribute("lineNo")
            inline = 1  # we’re inside a line
            open_elements.peek().appendChild(node)  # add as child of current node in output tree
            open_elements.push(node)  # keep track of open elements
        elif event == pulldom.END_ELEMENT and node.localName == 'line':
            inline = 0  # when we finish our work here, we’ll no longer be inside a line
            open_elements.pop()  # line is finished
            # serialize XML, strip declaration, rewrite &quot; entity as character
            result.append([int(stanzaNo), int(lineNo),
                WS_RE.sub(" " ,
                open_elements.pop().toxml().replace('<?xml version="1.0" ?>', '').replace('&quot;', '"'))])
        elif event == pulldom.START_ELEMENT and node.localName == 'stress':
            open_elements.peek().appendChild(node)  # add as child of current node in output tree
            open_elements.push(node)  # keep track of open elements
        elif event == pulldom.END_ELEMENT and node.localName == 'stress':
            open_elements.pop()  # stress element is finished
        elif event == pulldom.CHARACTERS and inline:  # keep text only inside lines
            t = d.createTextNode(node.data)
            open_elements.peek().appendChild(t)
    return result

## Parse the XML into an array of arrays

In [5]:
with open("data_samples/eo1.xml") as f:
    all_lines = process(f)
all_lines[:5]  # take a look

[[1,
  1,
  '<line lineNo="001">"Мой дядя самых честных пр<stress>а</stress>вил,</line>'],
 [1,
  2,
  '<line lineNo="002">Когда не в шутку занем<stress>о</stress>г,</line>'],
 [1, 3, '<line lineNo="003">Он уважать себя заст<stress>а</stress>вил</line>'],
 [1, 4, '<line lineNo="004">И лучше выдумать не м<stress>о</stress>г.</line>'],
 [1, 5, '<line lineNo="005">Его пример другим на<stress>у</stress>ка;</line>']]

## General descriptive information

Use `//` for integer division to return *54* instead of *54.0*.

In [6]:
line_count = len(all_lines)
print ('There are ' + str(line_count // 14) + ' 14-line stanzas in this sample, with a total of ' + 
       str(line_count) + ' lines.\nSince we know that the poem is fully rhymed, there are ' + 
       str(line_count // 2) + ' rhyme pairs in the sample.')

There are 54 14-line stanzas in this sample, with a total of 756 lines.
Since we know that the poem is fully rhymed, there are 378 rhyme pairs in the sample.


## Write the data into a dataframe

In [7]:
df = pd.DataFrame(all_lines, columns=["StanzaNo", "LineNo", "Text"])
df.head(5)

,StanzaNo,LineNo,Text
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<..."
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre..."
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress..."
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres..."
4,1,5,"<line lineNo=""005"">Его пример другим на<stress..."


## Transliterate all lines and save in new column

1. Because only the last stress in the line is marked, the phonetic representation of all words except the last is incorrect. That doesn’t matter for the analysis of end rhyme.
1. Words in foreign languages are not being treated specially, and are therefore usually phonetically incorrect. That *does* matter for the analysis of end rhyme. Deal with it later, first by excluding those lines (revise the XML parsing to record that information), and eventually by phoneticizing them correctly.
1. The `transliterate()` function is part of the [custom *cyr2phon* package](cyr2phon/cyr2phon.py).

In [8]:
trans_vec = np.vectorize(cyr2phon.transliterate)
df["Phonetic"] = trans_vec(df["Text"])
df.head()  # take a look

,StanzaNo,LineNo,Text,Phonetic
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<...",maJ DiDi samix Čistnix prAVil
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre...",kagda Nifšutku zaNimOk
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress...",an uvažaT SiBi zastAVil
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres...",iluČši vidumaT NimOk
4,1,5,"<line lineNo=""005"">Его пример другим на<stress...",Jiva pRiMir druGim naUka


## Write the rhyme word into a new column

In [9]:
df["RhymeWord"] = df["Phonetic"].str.split().str[-1] # clitics have already been joined
df.head()  # take a look

,StanzaNo,LineNo,Text,Phonetic,RhymeWord
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<...",maJ DiDi samix Čistnix prAVil,prAVil
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre...",kagda Nifšutku zaNimOk,zaNimOk
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress...",an uvažaT SiBi zastAVil,zastAVil
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres...",iluČši vidumaT NimOk,NimOk
4,1,5,"<line lineNo=""005"">Его пример другим на<stress...",Jiva pRiMir druGim naUka,naUka


## Identify rhyme zone and write into new column

The *rhyme zone* is the portion of the line that participates in line end-rhyme. According to Russian rhyming conventions, the rhyme zone typically begins with the last stressed vowel of the line and continues until the end of the line. The one exception is that open masculine rhyme (that is, rhyme involving stress on a final syllable that ends in a vowel, e.g., **себя́** *[SiBA]*) also requires a *supporting consonant*, that is, it also requires that the consonants *before the stressed vowels* (not otherwise considered part of the rhyme zone* also agree. For example:

* _see_ and _tree_ do not rhyme in Russian because this open (ends in a vowel sound) masculine (stress on the final syllable) rhyme does not have a supporting consonant (consonants before the stressed vowels do not agree).
* *seat* and *treat* do rhyme in Russian because closed (ends in a consonant sound) masculine (stress on the final syllable) rhyme does not require a supporting consonant, so the lack of phonetic correspondence between the consonants before the stressed vowels does not matter.

Russian rhyme may also be *enriched* by phonetic agreement or similarity outside the rhyme zone. For example, *stop* and *strop* constitute a perfect rhyme because the *op* sounds match. Nonetheless, the match of *st* before the rhyme zone enhances, or enriches, the rhyme. The present study ignores enrichment and concentrates only on the core rhyme components, but enrichment will be incorporated into the analysis at a later stage.

With that said, this first pass at identifying the rhyme zone removes the pretonic segments, but not the final consonant of a pretonic onset where a supporting consonant is not needed. More cleaning to follow!

In [12]:
rhymezonepat = re.compile(r'(.?[AEIOU]$)|([AEIOU].*$)')
def remove_pretonic_segments(s: str) -> str: # removes segments in place
    return rhymezonepat.search(s).group(0)
df["RhymeZone"] = df["RhymeWord"].apply(remove_pretonic_segments)
df.head(14)

,StanzaNo,LineNo,Text,Phonetic,RhymeWord,RhymeZone
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<...",maJ DiDi samix Čistnix prAVil,prAVil,AVil
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre...",kagda Nifšutku zaNimOk,zaNimOk,Ok
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress...",an uvažaT SiBi zastAVil,zastAVil,AVil
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres...",iluČši vidumaT NimOk,NimOk,Ok
4,1,5,"<line lineNo=""005"">Его пример другим на<stress...",Jiva pRiMir druGim naUka,naUka,Uka
5,1,6,"<line lineNo=""006"">Но, боже мой, какая ск<stre...",nabaži maJ kakaJi skUka,skUka,Uka
6,1,7,"<line lineNo=""007"">С больным сидеть и день и н...",zbaLnim SiDiT iDiN inOČ,inOČ,OČ
7,1,8,"<line lineNo=""008"">Не отходя ни шагу пр<stress...",NiatxaDi Nišagu prOČ,prOČ,OČ
8,1,9,"<line lineNo=""009"">Какое низкое ков<stress>а</...",kakaJi NiskaJi kavArstva,kavArstva,Arstva
9,1,10,"<line lineNo=""010"">Полу-живого забавл<stress>я...",palu-živava zabavLAT,zabavLAT,AT
